generate pseudowords, tokenize them, then sort tokenization types

In [3]:
import numpy as np 
import pandas as pd
from gibberish import Gibberish
from transformers import BertModel, BertConfig
from tqdm.notebook import tqdm
import os

ModuleNotFoundError: No module named 'gibberish'

In [ ]:
gib = Gibberish()

pseudowords = [] # list of pseudowords
pseudo_affixes = {} # dict of singular pseudoword keys, values are "s" or "es" (plural affix)
pseudo_articles = {} # dict of plural pseudoword keys, values are "el" or "la"

# wordsC = gib.generate_words(50) # words ending in consonants
# for word in wordsC:
    # pseudowords.append(word)
for i in range(7000):
    # add words ending in vowels - easiest to assign gender to 
    # end_vowel is not a viable arg for generate_words()
    pseudowords.append(gib.generate_word(end_vowel=True))
# print(pseudowords)
# print(len(pseudowords))

for i in range(len(pseudowords)):
    word = pseudowords[i]
    if word[-1] == "s":
        pseudowords[i] = word[:-1]

ok_vowels = ["a", "o"]
discards = []
for i in range(len(pseudowords)):
    word = pseudowords[i]
    if word[-1] not in ok_vowels:
        discards.append(word)
for word in pseudowords:
    if word in discards:
        pseudowords.remove(word)

pseudowords = [word for word in pseudowords if word not in discards]

print(pseudowords)

# populate dicts
for i in range(len(pseudowords)):
    word = pseudowords[i]
    pseudo_affixes[word] = "s" # same for all since they all end in vowels - this is probably not a necessary dict unless we end in Cs
    art_key = word + "s" # key for dict of articles is pluralized pseudoword
    if word[-1] == "a":
        pseudo_articles[art_key] = "la"
    elif word[-1] == "o":
        pseudo_articles[art_key] = "el"

print(pseudo_affixes)
print(pseudo_articles) # need to spit this up into tokenization-type dicts in the next block 



['sco', 'drua', 'pia', 'scha', 'schloo', 'zoia', 'speo', 'wraa', 'schreia', 'croo', 'mcgua', 'blao', 'mcfua', 'dweo', 'weia', 'schmeia', 'shea', 'shrio', 'dynao', 'staa', 'typaa', 'rhea', 'groo', 'stria', 'mcdo', 'zua', 'mcno', 'physiao', 'psychio', 'twoa', 'gliao', 'shaio', 'straa', 'mccroa', 'ghoo', 'ca', 'mcloia', 'mccoa', 'schwaia', 'sliao', 'mcnoa', 'yuo', 'ho', 'scoia', 'glao', 'chuo', 'rhoia', 'synea', 'droia', 'breia', 'luo', 'cycliao', 'fluo', 'skaia', 'mcaia', 'toia', 'psycheo', 'reia', 'yuo', 'dynua', 'joo', 'joo', 'hraio', 'cziao', 'raa', 'giao', 'gro', 'schleia', 'zo', 'mccuo', 'mccrea', 'frua', 'szuo', 'zaio', 'wroo', 'schloo', 'koa', 'slao', 'spraia', 'mcmia', 'zoo', 'gha', 'boo', 'spla', 'schneo', 'gnaa', 'psychia', 'soa', 'mccloa', 'blao', 'kwo', 'physa', 'hydreo', 'hydruo', 'schoia', 'bluo', 'chuo', 'troia', 'chroa', 'wio', 'physaio', 'hydreo', 'czo', 'thraio', 'saia', 'kloa', 'giao', 'sceia', 'zuo', 'clao', 'skio', 'physuo', 'keo', 'two', 'goia', 'treia', 'glaia', 'q

In [ ]:
import torch
from transformers import BertForMaskedLM, BertTokenizer

### Create the tokenizer and the model

tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model.eval()

pseudo_tokens = {} # dict of pluralized pseudowords and their tokens
for i in range(len(pseudowords)):
    word = pseudowords[i]
    word += pseudo_affixes[word] # pluralize it to tokenize it; add the affix on!
    tokens = tokenizer.tokenize(word)
    if word not in pseudo_tokens.keys():
        pseudo_tokens[word] = tokens
# print(pseudo_tokens)

singletok = {}
multi_morph = {}
multi_nonmorph = {}
for key, value in pseudo_tokens.items():
    if len(value) == 1:
        singletok[key] = value
    else:
        if value[-1] in ["##s", "##es"]:
            multi_morph[key] = value
        else:
            multi_nonmorph[key] = value

# these dicts are plural pseudowords and their tokens
print("single: ", singletok)
print("multi morph: ", multi_morph)
print("multi nonmorph: ", multi_nonmorph)

# these dicts are plural pseudowords and their singular articles, like the original paper has
# dct_noun_singletok = {}
# dct_noun_multi_morph = {}
# dct_noun_multi_nonmorph = {}

def populate_noun_dict(token_dict, article_dict):
    noun_dict = {}
    for key in token_dict.keys(): # singletok, multi_morph, or multi_nonmorph
        noun_dict[key] = article_dict[key]
    return noun_dict

dct_noun_singletok = populate_noun_dict(singletok, pseudo_articles)
dct_noun_multi_morph = populate_noun_dict(multi_morph, pseudo_articles)
dct_noun_multi_nonmorph = populate_noun_dict(multi_nonmorph, pseudo_articles)

print(dct_noun_singletok)


single:  {'cas': ['cas'], 'hos': ['hos'], 'gros': ['gros'], 'bos': ['bos'], 'cos': ['cos'], 'dos': ['dos'], 'reas': ['reas'], 'vos': ['vos'], 'fos': ['fos'], 'gras': ['gras'], 'pas': ['pas'], 'caos': ['caos'], 'sos': ['sos'], 'pros': ['pros'], 'bras': ['bras'], 'vas': ['vas'], 'sas': ['sas'], 'has': ['has'], 'ros': ['ros'], 'los': ['los'], 'gas': ['gas'], 'dias': ['dias'], 'bas': ['bas'], 'tas': ['tas'], 'mas': ['mas'], 'mos': ['mos'], 'plas': ['plas'], 'creas': ['creas'], 'tos': ['tos'], 'blas': ['blas'], 'clas': ['clas'], 'nas': ['nas'], 'tras': ['tras'], 'las': ['las'], 'pos': ['pos'], 'dios': ['dios'], 'nos': ['nos']}
multi morph:  {'zuas': ['z', '##ua', '##s'], 'twoas': ['t', '##w', '##oa', '##s'], 'gliaos': ['gl', '##iao', '##s'], 'ghoos': ['g', '##hoo', '##s'], 'mccoas': ['m', '##c', '##coa', '##s'], 'dynuas': ['d', '##yn', '##ua', '##s'], 'zos': ['zo', '##s'], 'koas': ['k', '##oa', '##s'], 'mcmias': ['m', '##c', '##mia', '##s'], 'schneos': ['s', '##ch', '##neo', '##s'], 'bluos'

need to make the nounlist files from the datasets folder on github


In [ ]:
### Define useful functions

def make_sentences_df(dct_noun,df_noun):
    """Creates sentence templates masking what should be a definite or indefinite, plural or singular article
    INPUTS: dct_noun: dictionary, noun:article pairs, the article is lowercase, singular, and gendered--this 
                      was hand-annotated from native Caribbean Spanish speaker intuition and checks in online
                      dictionary of the Real Academia Española; if there are any concerns about gendered article 
                      errors, check here first
            df_noun: pandas dataframe, cols: whole_word (plural), root (becomes our lemma), affix (s or es)--
                     this df is drawn from a noun subset of AnCora Tree Bank"""
    
    gather_df = []
    for noun in tqdm(dct_noun.keys()): 
        
        subdf = df_noun[df_noun['whole_word'] == noun]

        ### set up singular & plural template components
        sing_art_def = dct_noun[noun]
        sing_art_def = str(sing_art_def[0].upper() + sing_art_def[1])

        sing_subj = subdf['root'].values[0]

        plur_subj = noun

        ### set up target definite and indefinite articles
        if sing_art_def == 'El':

            plur_art_def = 'Los'
            plur_art_indef = 'Unos'

            sing_art_indef = 'Un'


        elif sing_art_def == 'La': 

            plur_art_def = 'Las'
            plur_art_indef = 'Unas'

            sing_art_indef = 'Una'


        ### set up composite template components
        affix = subdf['affix'].values[0]
        comp_subj = sing_subj +'##'+ affix


        ### create all sentence templates
        article_types = ['definite','indefinite']
        for article_type in article_types: 

            sing_sentence_template = '[MASK] ' + sing_subj 

            plur_sentence_template = '[MASK] ' + plur_subj 

            comp_sentence_template = '[MASK] ' + comp_subj 


            ### set up all the column/row entries you will store per lemma

            all_wordforms = [sing_subj,
                             plur_subj,
                             comp_subj]

            all_word_number = ['sing',
                               'plur',
                               'plur'] 

            all_n_tokens = [len(tokenizer.encode(sing_subj,add_special_tokens=False)),
                            len(tokenizer.encode(plur_subj,add_special_tokens=False)),
                            len(tokenizer.encode(sing_subj,add_special_tokens=False)) + len(tokenizer.encode(['##'+affix],add_special_tokens=False))
                           ]

            all_sentence_templates = [sing_sentence_template,
                                      plur_sentence_template,
                                      comp_sentence_template]
            
            #this is a super important column! distinguishes the default tokenization from 
            #our artificially imposed tokenization scheme
            all_tokenization_types = ['default',
                                      'default',
                                      'artificial'
                                     ]

            if article_type == 'definite': 
                sing_art = sing_art_def
                plur_art = plur_art_def

            elif article_type == 'indefinite': 
                sing_art = sing_art_indef
                plur_art = plur_art_indef

            d = {'lemma': np.repeat(sing_subj,len(all_sentence_templates)),
                 'word_form': all_wordforms,
                 'word_number': all_word_number,
                 'n_tokens': all_n_tokens,
                 'tokenization_type': all_tokenization_types,
                 'sentence': all_sentence_templates,
                 'target_ART_sing': sing_art,
                 'target_ART_plur': plur_art,
                 'article_type': article_type,
                 'affix': np.repeat(affix,len(all_sentence_templates))
                }

            gather_df.append(pd.DataFrame(d))

    sentence_df = pd.concat(gather_df,ignore_index=True)
    return sentence_df


### Model Predictions 

def find_sublist_index(list, sublist):
    """Find the first occurence of sublist in list.
    Return the start and end indices of sublist in list.
    Used to find index of [MASK] in template sentences.

    h/t GPT-3-codex for writing this."""

    for i in range(len(list)):
        if list[i] == sublist[0] and list[i:i+len(sublist)] == sublist:
            return i, i+len(sublist)
    return None


def get_article_predictions(df,data_source): 
    """Predict the likelihood of target definite/indefinite and singular/plural articles
       Will assume you've already loaded and defined your tokenizer and model.
       Will iterate row by row in your dataframe `df`, containing cols for masked sentences,
       the corresponding lemma and plural forms being tested. When it comes across a row
       with column `tokenizer_type` label `artificial`, will shunt you to a process that creates
       the inputs to the model by hand (if there are any issues/concerns with how the artificial
       tokenization proceeds and leads to predictions, check the following `if` statement: 
       `if row['tokenization_type'] == 'artificial'`)
       
       INPUTS: df, pandas dataframe, cols for lemma, word_number (plural, singular), tokenization_type
                   (artificial/default), masked sentence, target singular and plural articles to 
                   get probabilities for filling in the [MASK], and others"""

    gather_df = []
    gather_debug = []
    for (_,row) in tqdm(df.iterrows(),total=df.shape[0]):

        target_article_sing = row['target_ART_sing']
        target_article_plur = row['target_ART_plur']


        #tokens for each article type 
        token_article_sing = tokenizer.encode(target_article_sing,
                                              add_special_tokens=False
                                         )
        token_article_plur = tokenizer.encode(target_article_plur,
                                              add_special_tokens=False
                                         )
        #token for mask
        token_mask = tokenizer.encode('[MASK]',
                                      add_special_tokens=False
                                     )


        ### Set up your representation of the sentence for the 
        #.  model

        if row['tokenization_type'] == 'artificial': 

            if '##es' in row['word_form']:

                token_affix = tokenizer.convert_tokens_to_ids(["##es"])


            elif '##s' in row['word_form']: 

                token_affix = tokenizer.convert_tokens_to_ids(["##s"])


            #token for singular form
            lemma = row['lemma']
            token_lemma = tokenizer.encode(lemma,
                                           add_special_tokens=False
                                          )
            #TODO: How to combine the two tokens into one if the source is the non-morphemic plurals
 
            #token for special start
            start = '[CLS]'
            token_start = tokenizer.encode(start,
                                           add_special_tokens=False)
            

            #token for special end
            ending = '[SEP]'
            token_end = tokenizer.encode(ending,
                                         add_special_tokens=False)
            
            
            ### Collect your tokens into a list that you will then flatten
            #   prior to converting to tensor
            bulky_token_list = [token_start,
                                token_mask,
                                token_lemma,
                                token_affix,
                                token_end
                                ]
            flat_token_list = [item for sublist in bulky_token_list for item in sublist]
            token_idx = torch.tensor([flat_token_list])
            

            inputs = {'input_ids': token_idx,
                      'token_type_ids': torch.zeros_like(token_idx),
                      'attention_mask': torch.ones_like(token_idx)
                     }

        elif row['tokenization_type']=='default': 

            inputs = tokenizer(row['sentence'],
                               return_tensors='pt',
                               add_special_tokens=True
                              )
            
        model_token_inputs = tokenizer.convert_ids_to_tokens(inputs['input_ids'].tolist()[0])
        model_token_inputs = ' , '.join(model_token_inputs)

        ### Predict the item that should fill in the mask!

        outputs = model(**inputs)

        #find the index of the mask in sentence
        midx = find_sublist_index(inputs["input_ids"][0].tolist(),
                                  token_mask)
                                 
        masked_token_logits = outputs.logits[0][midx[0]]
        masked_token_probs = torch.softmax(masked_token_logits, dim=0)


        prob_article_sing = masked_token_probs[token_article_sing].item()
        prob_article_plur = masked_token_probs[token_article_plur].item()

        prob_list = [prob_article_sing, prob_article_plur]
        article_list = [target_article_sing, target_article_plur]
    

    
        ### Store your results

        d = {'lemma': np.repeat(row['lemma'],len(prob_list)),
             'word_form': np.repeat(row['word_form'],len(prob_list)),
             'word_number': np.repeat(row['word_number'],len(prob_list)),
             'n_tokens': np.repeat(row['n_tokens'],len(prob_list)),
             'tokenization_type': np.repeat(row['tokenization_type'],len(prob_list)),
             'article_probs': prob_list,
             'article_number': ['singular','plural'],
             'article_type': np.repeat(row['article_type'],len(prob_list)),
             'article': article_list,
             'affix': row['affix'],
             'sentence': row['sentence'],
             'model_token_inputs': model_token_inputs,
             'source': data_source
            }
        gather_df.append(pd.DataFrame(d))


        debug_d = {'lemma': [row['lemma']],
                   'word_form': [row['word_form']],
                   'tokenized_sentence': [inputs["input_ids"][0].tolist()],
                   'mask_index': [midx[0]]
                  }
        gather_debug.append(pd.DataFrame(debug_d))


    probs_df = pd.concat(gather_df,ignore_index=True)
    debug_df = pd.concat(gather_debug,ignore_index=True)
    
    return probs_df,debug_df

In [ ]:
# For default SINGLE-TOKEN plurals

df_noun_s = pd.DataFrame(columns=['whole_word', 'root', 'affix'])
df_noun_s['whole_word'] = dct_noun_singletok.keys()
s_root_list = []
for key in dct_noun_singletok.keys():
    s_root_list.append(key[:-1])
s_affix_list = []
for i in range(len(s_root_list)):
    s_affix_list.append("s")
df_noun_s['root'] = s_root_list
df_noun_s['affix'] = s_affix_list
# print(df_noun_s.head())

dct_noun_s = dct_noun_singletok

df_singletok = make_sentences_df(dct_noun_s,df_noun_s)

print('original list len:',df_noun_s.shape[0],'select list len:', len(dct_noun_s))

data_source = 'single-token' # CHECK THIS!
probs_df_singletok,_ = get_article_predictions(df_singletok,data_source)


# # For default MULTI-TOKEN, MORPHEMIC plurals

df_noun_mm = pd.DataFrame(columns=["POS", "whole_word", "root", "affix"])
df_noun_mm['whole_word'] = dct_noun_multi_morph.keys()
mm_root_list = []
for key in dct_noun_multi_morph.keys():
    mm_root_list.append(key[:-1])
mm_affix_list = []
mm_pos_list = []
for i in range(len(mm_root_list)):
    mm_affix_list.append("s")
    mm_pos_list.append("NOUN")
df_noun_mm['POS'] = mm_pos_list
df_noun_mm['root'] = mm_root_list
df_noun_mm['affix'] = mm_affix_list

dct_noun_mm = dct_noun_multi_morph
df_multitok_morph = make_sentences_df(dct_noun_mm,df_noun_mm)

print('original list len:',df_noun_mm.shape[0],'select list len:', len(dct_noun_mm))

data_source = 'morphemic' 
probs_df_multitok_morph,_ = get_article_predictions(df_multitok_morph,data_source)

# # For default MULTI-TOKEN, NONMORPHEMIC plurals

df_noun_nm = pd.DataFrame(columns=["whole_word", "root", "affix", "tokenized"])
df_noun_nm['whole_word'] = dct_noun_multi_nonmorph.keys()
nm_root_list = []
for key in dct_noun_multi_nonmorph.keys():
    nm_root_list.append(key[:-1])
nm_affix_list = []
nm_tokenized_list = []
for i in range(len(nm_root_list)):
    word = nm_root_list[i] + "s"
    nm_affix_list.append("s")
    nm_tokenized_list.append(dct_noun_multi_nonmorph[word])
    # mm_pos_list.append("NOUN")
df_noun_nm['root'] = nm_root_list
df_noun_nm['affix'] = nm_affix_list
df_noun_nm['tokenized'] = nm_tokenized_list

dct_noun_nm = dct_noun_multi_nonmorph
df_multitok_nonmorph = make_sentences_df(dct_noun_nm,df_noun_nm)

print('original list len:',df_noun_nm.shape[0],'select list len:', len(dct_noun_nm))
print(df_noun_nm.head())

data_source = 'nonmorphemic' 
probs_df_multitok_nonmorph,_ = get_article_predictions(df_multitok_nonmorph,data_source)

  0%|          | 0/37 [00:00<?, ?it/s]

original list len: 37 select list len: 37


  0%|          | 0/222 [00:00<?, ?it/s]

  0%|          | 0/212 [00:00<?, ?it/s]

original list len: 212 select list len: 212


  0%|          | 0/1272 [00:00<?, ?it/s]

  0%|          | 0/1142 [00:00<?, ?it/s]

original list len: 1142 select list len: 1142
  whole_word    root affix tokenized
0       scos     sco     s        el
1      druas    drua     s        la
2       pias     pia     s        la
3      schas    scha     s        la
4    schloos  schloo     s        el


  0%|          | 0/6852 [00:00<?, ?it/s]

In [ ]:
probs_df_singletok['surprisal'] = probs_df_singletok['article_probs'].apply(lambda x: -np.log(x))
probs_df_multitok_morph['surprisal'] = probs_df_multitok_morph['article_probs'].apply(lambda x: -np.log(x))
probs_df_multitok_nonmorph['surprisal'] = probs_df_multitok_nonmorph['article_probs'].apply(lambda x: -np.log(x))

In [ ]:
### Pause to check that the default tokenizations for plurals are, in fact, morphemic when you expect them to be

non_morphemic_list = []
for (_,row) in df_multitok_morph.iterrows(): 
    
    sing = row['lemma']
    plur = row['word_form']
    affix = row['affix']
    mod_affix = "##" + affix
    
    sing_tokids = tokenizer.encode(sing,add_special_tokens=False)
    
    if row['tokenization_type'] == 'artificial': 
        plur_tokids = tokenizer.encode([sing]+[mod_affix],add_special_tokens=False)
        
    elif row['tokenization_type'] == 'default': 
        plur_tokids = tokenizer.encode(plur,add_special_tokens=False)


    affix_tokid = tokenizer.encode([mod_affix],add_special_tokens=False)[0]

    n_plural_tokens = len(plur_tokids)
    
    sing_tokens = tokenizer.convert_ids_to_tokens(sing_tokids)
    plural_tokens = tokenizer.convert_ids_to_tokens(plur_tokids)
    
    if n_plural_tokens == 1:
        check_morph = "singular single"
        
    elif affix_tokid in plur_tokids:
        check_morph = "morphemic"

    ### Multi-token, non-morphemic
    else:
        check_morph = "non_morphemic"

        print(sing,plural_tokens,check_morph)
        
        non_morphemic_list.append(sing)

print(non_morphemic_list)

zua ['z', '##ua'] non_morphemic
zua ['z', '##ua'] non_morphemic
twoa ['t', '##w', '##oa'] non_morphemic
twoa ['t', '##w', '##oa'] non_morphemic
gliao ['gl', '##iao'] non_morphemic
gliao ['gl', '##iao'] non_morphemic
ghoo ['g', '##hoo'] non_morphemic
ghoo ['g', '##hoo'] non_morphemic
mccoa ['m', '##c', '##coa'] non_morphemic
mccoa ['m', '##c', '##coa'] non_morphemic
dynua ['d', '##yn', '##ua'] non_morphemic
dynua ['d', '##yn', '##ua'] non_morphemic
koa ['k', '##oa'] non_morphemic
koa ['k', '##oa'] non_morphemic
mcmia ['m', '##c', '##mia'] non_morphemic
mcmia ['m', '##c', '##mia'] non_morphemic
schneo ['s', '##ch', '##neo'] non_morphemic
schneo ['s', '##ch', '##neo'] non_morphemic
bluo ['bl', '##uo'] non_morphemic
bluo ['bl', '##uo'] non_morphemic
zuo ['z', '##uo'] non_morphemic
zuo ['z', '##uo'] non_morphemic
qoa ['q', '##oa'] non_morphemic
qoa ['q', '##oa'] non_morphemic
sruo ['sr', '##uo'] non_morphemic
sruo ['sr', '##uo'] non_morphemic
spoa ['sp', '##oa'] non_morphemic
spoa ['sp', '#

In [ ]:
### Pause to check that the default tokenizations for plurals are, in fact, nonmorphemic when you expect them to be

morphemic_list = []
for (_,row) in df_multitok_nonmorph.iterrows(): 
    
    sing = row['lemma']
    plur = row['word_form']
    affix = row['affix']
    mod_affix = "##" + affix
    
    sing_tokids = tokenizer.encode(sing,add_special_tokens=False)
    
    if row['tokenization_type'] == 'artificial': 
        plur_tokids = tokenizer.encode([sing]+[mod_affix],add_special_tokens=False)
        
    elif row['tokenization_type'] == 'default': 
        plur_tokids = tokenizer.encode(plur,add_special_tokens=False)


    affix_tokid = tokenizer.encode([mod_affix],add_special_tokens=False)[0]

    n_plural_tokens = len(plur_tokids)
    
    sing_tokens = tokenizer.convert_ids_to_tokens(sing_tokids)
    plural_tokens = tokenizer.convert_ids_to_tokens(plur_tokids)
    
    if n_plural_tokens == 1:
        check_morph = "singular single"
        
    elif (affix_tokid in plur_tokids) & (row['tokenization_type']=='default'):
        check_morph = "morphemic"
        
        morphemic_list.append(sing)
        print(sing,plural_tokens,check_morph)


    ### Multi-token, non-morphemic
    else:
        check_morph = "non_morphemic"
        
print(morphemic_list)

[]


In [ ]:
### Save each of these to a dataframe!

savepath = 'results_pseudo_article-agreement/'

if not os.path.exists(savepath): 
    os.mkdir(savepath)
    

probs_df_singletok.to_csv(os.path.join(savepath,'results_singletok.csv'))
probs_df_multitok_morph.to_csv(os.path.join(savepath,'results_multitok_morph.csv'))
probs_df_multitok_nonmorph.to_csv(os.path.join(savepath,'results_multitok_nonmorph.csv'))